In [ ]:
import data, utils, processing, metric
import numpy as np
import pandas as pd
import pingouin as pg

root = 'E:/LIMITLESS_DL/Respiration/'
data_root = f"{root}DATA/"
result_root = f"{root}RESULTS/"

treatment = "STATIC"
breath = "Breathhold"

patient_path, num_fx = data.patient_path(data_root, treatment, breath)
patient_ID = patient_path.split("/")[5].split("_")[0]

patient_path, num_fx = data.patient_path(data_root, treatment, breath)

fx_levels, fx_errors = [], []
for fx in range(1, num_fx+1):
    fraction_path, num_fld = data.fraction_path(patient_path, fx)
    fld_levels, fld_errors = [], []

    for field in range(1, num_fld+1):
        (data_Times, data_Amps), (beam_Times, beam_Amps) = data.read_field_data(fraction_path, field)
        cutted_Amps = processing.cut_by_beams(data_Times, data_Amps, beam_Times)
        enabled_intervals, num_intvs = processing.beam_enabling_intervals(data_Times, data_Amps, beam_Times)
        beam_levels, beam_errors = [], []

        for intv in range(num_intvs):
            intv_level = metric.avg_lvl_per_interval(enabled_intervals[intv])
            intv_error = metric.error_per_interval(enabled_intervals[intv])
            beam_levels.append(intv_level)
            beam_errors.append(intv_error)
        
        fld_levels.append(np.mean(beam_levels))
        fld_errors.append(np.mean(beam_errors))

    fx_levels.append(np.mean(fld_levels))
    fx_errors.append(np.mean(fld_errors))

print(len(fx_levels))
print(len(fx_errors))

10
10


In [17]:
def icc(scores):
    # Calculate number of raters (n_raters) and subjects (n_subjects)
    n_raters = 5
    n_subjects = 2

    scores_matrix = np.array(scores).reshape(n_subjects, n_raters)

    # Mean calculations
    mean_total = np.mean(scores_matrix)  # Overall mean
    mean_subject = np.mean(scores_matrix, axis=1)

    # Variance components based on ANOVA
    MS_subject = np.sum((mean_subject - mean_total) ** 2) * n_raters / (n_subjects - 1)
    MS_residual = np.sum((scores_matrix - mean_subject[:, None]) ** 2) / (n_raters * (n_subjects - 1))

    # ICC(1) calculation
    icc_1 = (MS_subject - MS_residual) / (MS_subject + (n_raters - 1) * MS_residual)

    print(f"Intraclass Correlation Coefficient ICC(1): {icc_1:.4f}")
    return